# The Heston Model

**2025 Introduction to Quantiative Methods in Finance**

**The Erdös Institute**

In [ ]:
#Package Import
import numpy as np
import pandas as pd
import yfinance as yf
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy.stats import shapiro
import scipy.stats as stats
from scipy.optimize import brentq
from mpl_toolkits.mplot3d import Axes3D
sns.set_style('darkgrid')


%run functions_black_scholes.py


###Summary of Delta Hedging Sold Call Options

We looked into selling call options, assuming the distribution of stock paths follows a Geometric Brownian Motions, and the hedging with respect to the underlying stock until the expiration of the contract. There are two issues with this:

1) Geometric Brownian Motion assumes a static volatility.


2) It may be beneficial to sell or buy back an option before the expiration. It is therefore to understand the dynamics of not only the stock price, but the prices of option contracts in the market.

### Exploration

We will start by examining some historical option prices on the market and examine what the prices reflect.


One will quickly notice that market prices of options do not reflect prices that would be achieved by pricing according to the Black-Scholes equation.

**Recall**: The price of a stock option is a function of the stock's spot price, the strike price of the contract, and a constant volatility. 

If we examine the market prices of an option contract, the only unknown is the volatility.

**Implied Volatility** The implied volatility of an option contract with strike price $K$, time-to-expiration $t$, and premium $P$, is the $\sigma$ so that the Black-Scholes value of the option with volatility constant $\sigma$ is $P$. 

In [ ]:
def implied_volatility_call(market_price, S0, K, t, r, sigma_bounds=(1e-6, 2)):
    def objective(sigma):
        return bs_call(S0, K, sigma, t, r) - market_price

    try:
        return brentq(objective, *sigma_bounds)
    except ValueError:
        return np.nan


In [ ]:
##Test implied volatility function

# Parameters
S0 = 550     
K = 545     
t = 1      
r = 0.035     
premium = 95.23

sigma = implied_volatility_call(premium, S0, K, t, r)
print(f"Implied Volatility of call options with market price ${premium}: {sigma}")


print('-----'*10)
print('-----'*10)

print(f'Black-Scholes price of call option with volatility {sigma}: ${bs_call(S0,K,sigma,t,r):.2f}')


In [ ]:
### Some historical market price data 
### Source: https://www.kaggle.com/datasets/zeeshanfirdousi/optionsstocksdata?resource=download
### Remark: Accurate and reliable market prices of options are not free.
### There are numerous monthly subscription services with reliable API's that provide minute-by-minute historical
### market option prices.


options_data = pd.read_csv('Options_Stocks_Data.csv')
options_data.sample(5)

In [ ]:
ibm_stock = yf.download('IBM', start = '2023-10-25', end = '2024-10-26')

plt.figure(figsize = (9,6))
plt.title('IBM stock', size = 20)

plt.plot(ibm_stock['Close'])


plt.show()

In [ ]:
S0 = ibm_stock['Close'].iloc[-1].iloc[0]


sigma = np.log(ibm_stock['Close']/ibm_stock['Close'].shift(1)).std().iloc[0]*np.sqrt(252)
r = 0.035


start_date = datetime.datetime(2024,10,25,3,0)


print(f'Value of IBM stock on 10-25-2024: ${S0:.2f}')
print('----'*10)
print('----'*10)
print(f'Historical volatility of IBM stock: {sigma}')

In [ ]:
options_data_calls = options_data[(options_data['type']=='call')]
options_data_calls = options_data_calls[['expiration', 'strike', 'mark']]

In [ ]:
def find_tte(expiration_date,start_date):
    '''returns time measured in years as a float between two dates
    
    Inputs:
    expiration_date (str): 'YYYY-MM-DD'
    current_date (datetime.datetime)
    
    Returns:
    Float of time to expiration in years
    '''
    ###Sets expiration time to 2:30 central = 3:30 eastern time = approx time when expiring options are settled
    tte = (datetime.datetime.strptime(expiration_date + '-2-30', '%Y-%m-%d-%H-%M')\
    - start_date).total_seconds()/(60*60*24*365)
    
    return tte

In [ ]:
options_data_calls['time_to_expiration'] = options_data_calls['expiration'].apply(
    lambda x: find_tte(x, start_date)
)


options_data_calls['implied_volatility'] = options_data_calls.apply(
    lambda row: implied_volatility_call(
        market_price=row['mark'],
        S0=S0,
        K=row['strike'],
        t=row['time_to_expiration'],
        r=r
    ),
    axis=1
)

In [ ]:
options_data_calls

In [ ]:
options_data = pd.read_csv('Options_Stocks_Data.csv')

options_data_calls = options_data[(options_data['type']=='call')]
options_data_calls = options_data_calls[['expiration', 'strike', 'mark']]


options_data_calls['time_to_expiration'] = options_data_calls['expiration'].apply(
    lambda x: find_tte(x, start_date)
)

options_data_calls = options_data_calls[(options_data_calls['time_to_expiration']>=1/12)\
                                       & (np.abs(options_data_calls['strike']-S0)/S0<=.2)\
                                       & (options_data_calls['time_to_expiration']<=1)]



options_data_calls['implied_volatility'] = options_data_calls.apply(
    lambda row: implied_volatility_call(
        market_price=row['mark'],
        S0=S0,
        K=row['strike'],
        t=row['time_to_expiration'],
        r=r
    ),
    axis=1
)


In [ ]:
for date in np.unique(options_data_calls['expiration'].values):
    options_with_same_expiration =  options_data_calls[options_data_calls['expiration']==date].copy()
    tte = options_with_same_expiration['time_to_expiration'].iloc[0]
    plt.figure(figsize = (10,8))
    plt.plot(options_with_same_expiration['strike'], options_with_same_expiration['implied_volatility'])

    plt.title(f'Market Volatility Smile Options with time to expiration: {tte:4f} years', size = 18)

    plt.ylabel('Implied Volatility', size = 15)
    plt.xlabel('Strike', size = 15)

    plt.axvline(S0, label = 'Spot Price of IBM', ls = '--', color = 'red')

    plt.axhline(sigma, label = 'Historical Volatility', ls = '--', color = 'black')

    plt.legend(fontsize = 12)

    plt.show()

In [ ]:
surface_df = options_data_calls.pivot_table(
    index='time_to_expiration', 
    columns='strike', 
    values='implied_volatility'
)



T, K = np.meshgrid(surface_df.index.values, surface_df.columns.values, indexing='ij')
IV = surface_df.values 
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(K, T, IV, cmap='plasma', edgecolor='k')
ax.set_xlabel('Strike Price', fontsize = 15)
ax.set_ylabel('Time to Expiration (Years)', fontsize = 15)
ax.set_zlabel('Implied Volatility', fontsize=15, labelpad=-30)
ax.set_title('Market Implied Volatility Surface', size = 20)



plt.tight_layout()
plt.show()


## The Heston Model:

Benefits:

1) Allows for non-constant volatility and incorporates an autoregressive feature in volatility modeling.

    a) However, tuning a Heston model using stock-data to simulate stock path movements is somewhat unreliable. There are too many parameters to estimate numerically!
    b) The Heston-model really shines as it often does a decent job of simulating the market prices of option contract as time passes.

2) The prices of option contracts under the assumption the stock path is modeled by the Heston model do not have a closed form solution, but standard methods of numerical integration yield quick numerical solutions.

3) The parameters of a Heston model can be tuned to best fit current market prices of option contracts. This allows a trader to model the distribution of a stock as predicted by the option market.

4) Consequently, a trader whose portfolio involves option trading may benefit from the use of the Heston model to better understand the distribution of their portfolio value before the expiration of options.

### The $1$-step Discrete Heston Stochastic Volatility Model

Let $0<t$ be an interval of time. The $1$-step discrete Heston model assumes the variance of the log-returns at time $t$ is modeled as

$$v_t = |v_0 + \kappa(\theta - v_0)t + \xi\sqrt{v_0t}\mathcal{N}^{v}(0,1)|$$

and the distribution of stock paths from time $0$ to $t$ is modeled as

$$S_t = S_0e^{(\mu + r - .5v_0)t + \sqrt{|v_0|t}*\mathcal{N^S(0,1)}}$$

where:

- $S_0$ is the initial stock price;
- $S_t$ is the stock price at time $t$;
- $v_0$ is the variance of the stock at time $0$;
- $v_t$ is the variance of the stock at time $t$;
- $\mu$ is the excess drift of the log-returns of the stock stock;
- $r$ is the risk-free interest rate;
- $\kappa$ is the **mean-reversion rate** of the variance process;
- $\theta$ is the **long-run variance level** of the variance process;
- $\xi$ is the **volatility of volatility**
- $\mathcal{N}^{v}(0,1)$ and $\mathcal{N}^{S}(0,1)$ are standard normal distributions with correlation $\rho$:

$$\rho = \mbox{corr}\left(\mathcal{N}^{v}(0,1),\mathcal{N}^{S}(0,1)\right) = \frac{\mbox{cov}(E\left[\mathcal{N}^{v}(0,1)\mathcal{N}^{S}(0,1))\right]}{\mbox{std}(\mathcal{N}^{v}(0,1))\mbox{std}(\mathcal{N}^{S}(0,1))} = E\left[\mathcal{N}^{v}(0,1)\mathcal{N}^{S}(0,1))\right].$$



**Remark**: If $N_1$ and $N_2$ are independent standard normal distributions and $-1\leq \rho\leq 1$, then 

$$Y = \rho N_1 + \sqrt{1-\rho^2}N_2$$

is a standard normal distribution so that $N_1$ and $Y$ have correlation $\mbox{corr}(N_1, Y) = \rho$.



### The general discrete Heston Model:

Let $0 = t_0<t_1<\cdots <t_n = t$, then the $n$-step discrete Heston model assumes for each $1\leq i\leq n$ the variance of the log-returns at time $t_i$ is modeled as

$$v_{t_i} = |v_{i-1} + \kappa(\theta - v_{t_{i-1}})(t_i-t_{i-1}) + \xi\sqrt{v_{i-1}(t_i-t_{i-1})}\mathcal{N}^{v}(0,1)|$$

and the distribution of stock paths from time $t_{i-1}$ to $t_i$ is modeled as

$$S_{t_i} = S_{t_{i-1}}e^{(\mu + r - .5v_0)(t_i-t_{i-1}) + \sqrt{|v_0|(t_i-t_{i-1})}\mathcal{N}^S(0,1)}$$.


### Remark:

The general definition of the Heston Model describes the variance and stock paths as being modeled as intertwined solutions to a system of stochastic Partial Differential Equations. The limiting distribution of variances and stock paths of the discrete models as the number of steps tends to $\infty$ is the continuous model described by the system of stochastic partial differential equations.

In [ ]:
kappa = 4
theta = .45**2
xi = .5
v0 = 0.4**2
rho = -0.05
S0 = 100
t = 1/12
steps = 21
r = 0.04
n_sims = 100000
dt = t / steps

# Heston simulation


#initialize paths
paths = 
vols = 
paths[:, 0] = 
vols[:, 0] = 

for i in range(steps):
    #Create Random noise at each step (this can be vectorized before the look to make things faster)

    vols[:, i+1] = 
    paths[:, i+1] = 

    
    
    
# Option pricing
strikes = range(int(.8*S0), int(1.2*S0))
S_t = paths[:, -1]
call_dict = {K: np.mean(np.maximum(S_t - K, 0)) * np.exp(-r * t) for K in strikes}


# Compute implied vols
implied_vol = {K: implied_volatility_call(call_dict[K], S0, K, t, r) for K in strikes}

# Plot
plt.figure(figsize = (10,8))
plt.plot(list(implied_vol.keys()), list(implied_vol.values()), marker='o')
plt.xlabel("Strike", size = 15)
plt.ylabel("Implied Volatility", size = 15)
plt.title("Implied Volatility Smile from Heston Simulation", size = 20)
plt.grid(True)
plt.axvline(S0, label= f'Spot Price: {S0}', color = 'red')

plt.legend(fontsize = 15)
plt.show()